In [ ]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.9.0-py3-none-any.whl size=203107 sha256=9c19f0070293fe55f793160e1ea871f3f510f00f3df7e170221046eb21ca0103
  Stored in directory: /root/.cache/pip/wheels/95/a7/fd/f667b71e29e3f64f239f30195efc39de642e72f8a47a6fcd8d
Successfully built pyTelegramBotAPI


In [248]:
import telebot

In [249]:
import pandas as pd
import random
import re

In [250]:
bot = telebot.TeleBot("5620973682:AAFtNyEVl9GcyivIONmKa9gVQbLz_82bT58")

In [251]:
token = "5620973682:AAFtNyEVl9GcyivIONmKa9gVQbLz_82bT58"

In [252]:
costo_estandar_por_envio = float(2.75)

In [253]:
from telebot.types import InlineKeyboardButton
from telebot.types import InlineKeyboardMarkup
from telebot.types import ForceReply
from telebot.types import ReplyKeyboardMarkup
from datetime import datetime
BOT_TOKEN = token
bot = telebot.TeleBot(BOT_TOKEN)
usuarios={}
markup4=InlineKeyboardMarkup(row_width=1)
b3=InlineKeyboardButton("VOLVER AL MENU PRINCIPAL",callback_data="salir")
markup4.add(b3)

letras=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
numeros=["1","2","3","4","5","6","7","8","9","0"]

In [254]:
from telebot.types import InlineKeyboardButton
from telebot.types import InlineKeyboardMarkup
from telebot.types import ForceReply
from telebot.types import ReplyKeyboardMarkup

In [255]:
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, "Bienvenido al chatbot. Por favor, selecciona una opción del menú:")
    bot.send_message(message.chat.id, "1. Gestor de citas")
    bot.send_message(message.chat.id, "2. Envío de paquetes")
    bot.send_message(message.chat.id, "3. Cotización de presupuestos")
    bot.send_message(message.chat.id, "4. Contactar con agente")
    bot.send_message(message.chat.id, "5. Salir")


In [256]:
@bot.message_handler(content_types=['text'])
def main_menu(message):
    if message.text == "1":
         bot.send_message(message.chat.id, "Bienvenido al gestor de citas")
         cmd_citas(message)
    elif message.text == "2":
        bot.send_message(message.chat.id, "Has seleccionado la opción 1. Por favor, selecciona una opción del submenú:")
        bot.send_message(message.chat.id, "1) Estado de paquete")
        bot.send_message(message.chat.id, "2) trascurso de paquete")
        bot.send_message(message.chat.id, "3) Nomdre de trasportista")
        bot.send_message(message.chat.id, "2) Salir al menu principal")
        bot.register_next_step_handler(message, envios_submenu)
    elif message.text == "3":
        bot.send_message(message.chat.id, "Cotización de presupuestos. ¿Qué desea consultar?")
        bot.send_message(message.chat.id, "1) ¿Cuál es el costo por envío?       3) ¿Qué métodos de pago tenemos?")
        bot.send_message(message.chat.id, "2) ¿Cuál es el costo por dimensiones de paquete de envío?        4) Volver al menú principal.")
        cot_menu(message)
    elif message.text == "4":
        contact_agent()
    elif message.text == "5":
        bot.send_message(message.chat.id, "Hasta luego vuelva pronto")
    else:
        bot.send_message(message.chat.id, "Por favor, selecciona una opción válida del menú.")


In [257]:
from sre_constants import NOT_LITERAL
@bot.message_handler(commands=["citas"])
def cmd_citas(message):
  bot.send_message(message.chat.id, "<b>AGENDADOR DE CITAS</b>"+"\n"+ "Favor de determinar el motivo de su cita")
  markup=InlineKeyboardMarkup(row_width=2)
  b1= InlineKeyboardButton("Citas para retiro",callback_data="retiro")
  b2= InlineKeyboardButton("Citas para envio",callback_data="envio")
  markup.add(b1,b2)
  bot.send_message(message.chat.id,"Tipos de Citas",reply_markup=markup)

@bot.callback_query_handler(func=lambda x:True)  
def feedback_button_inline(call):
  cid=call.from_user.id
  mid=call.message.id
  if call.data =="retiro":
    bot.send_message(cid, "Ejecuta el siguiente comando /retiro")
    
  if call.data =="envio":
    bot.send_message(cid, "Ejecuta el siguiente comando /envio")

@bot.message_handler(commands=["retiro"])
def cmd_retiro(message):
  markup=ForceReply()
  msg=bot.send_message(message.chat.id,"Digite su nombre completo",reply_markup=markup)

  bot.register_next_step_handler(msg,preguntar_num)

def preguntar_num(message):
  if message.text.isdigit():
    msg=bot.send_message(message.chat.id,"ESCRIBA UN NOMBRE VALIDO")
    bot.register_next_step_handler(msg,preguntar_num)
  else:
    usuarios[message.chat.id]={}
    usuarios[message.chat.id]["nombre"]=message.text
    markup=ForceReply()
    msg=bot.send_message(message.chat.id,"Digite su numero de afiliado",reply_markup=markup)
    bot.register_next_step_handler(msg,fechas_disponible)

def fechas_disponible(message):
  if message.text.isdigit():
    usuarios[message.chat.id]["num_afiliado"]=message.text
    markup2=ReplyKeyboardMarkup(
        one_time_keyboard=True,
        input_field_placeholder="Pulsar un boton",
        resize_keyboard=True
    )
    markup2.add("SI","NO")
    msg=bot.send_message(message.chat.id,"Desea generar una fecha disponible para retirar su paquete",reply_markup=markup2)
    bot.register_next_step_handler(msg,registro_retiro)
  else:
    msg=bot.send_message(message.chat.id,"NUMERO DE AFILIADO DESCONOCIDO \n Por Favor introducir uno valido")
    bot.register_next_step_handler(msg,fechas_disponible)
def registro_retiro(message):
    if message.text!="SI" and message.text!="NO":
       msg=bot.send_message(message.chat.id,"PRESIONE UN BOTÓN")
       bot.register_next_step_handler(msg,registro_envio)
    if message.text=="SI":
       
       inicio = datetime(2022, 1, 24)
       final =  datetime(2022, 5, 24)
       random_date = inicio + (final - inicio) * random.random()
       usuarios[message.chat.id]["fecha"]=random_date
       bot.send_message(message.chat.id,"<b>Su cita para retiro de mercancia a sido agendado</b>",parse_mode="HTML")  
       bot.send_message(message.chat.id,f"Fecha programada: {random_date}",reply_markup=markup4)   
                
    if message.text=="NO":
       bot.send_message(message.chat.id,"<b>Su cita para retiro de mercancia a sido cancelado</b>",parse_mode="HTML") 
usuarios=pd.DataFrame(usuarios) 

@bot.message_handler(commands=["envio"])
def cmd_envio(message):
  markup=ForceReply()
  msg=bot.send_message(message.chat.id,"Digite su nombre completo")

  bot.register_next_step_handler(msg,preguntar_num2)

def preguntar_num2(message):
  if message.text.isdigit():
    msg=bot.send_message(message.chat.id,"ESCRIBA UN NOMBRE VALIDO")
    bot.register_next_step_handler(msg,preguntar_num2)
  else:
    usuarios[message.chat.id]={}
    usuarios[message.chat.id]["nombre"]=message.text
    markup=ForceReply()
    msg=bot.send_message(message.chat.id,"Digite su numero de afiliado",reply_markup=markup)
    bot.register_next_step_handler(msg,fechas_disponible2)

def fechas_disponible2(message):
  if message.text.isdigit():
    usuarios[message.chat.id]["num_afiliado"]=message.text
    markup2=ReplyKeyboardMarkup(
        one_time_keyboard=True,
        input_field_placeholder="Pulsar un boton",
        resize_keyboard=True
    )
    markup2.add("SI","NO")
    msg=bot.send_message(message.chat.id,"Desea generar una fecha disponible para enviar su paquete",reply_markup=markup2)
    bot.register_next_step_handler(msg,registro_envio)
  else:
    msg=bot.send_message(message.chat.id,"NUMERO DE AFILIADO DESCONOCIDO \n Por Favor introducir uno valido")
    bot.register_next_step_handler(msg,fechas_disponible2)
def registro_envio(message):
    if message.text!="SI" and message.text!="NO":
       msg=bot.send_message(message.chat.id,"PRESIONE UN BOTÓN")
       bot.register_next_step_handler(msg,registro_envio)
    if message.text=="SI":
       
       inicio = datetime(2022, 1, 24)
       final =  datetime(2023, 1, 24)
       random_date = inicio + (final - inicio) * random.random()
       usuarios[message.chat.id]["fecha"]=random_date
       bot.send_message(message.chat.id,"<b>Su cita para envio de mercancia a sido agendado</b>",parse_mode="HTML")  
       bot.send_message(message.chat.id,f"Fecha programada: {random_date}",reply_markup=markup4)   
                
    if message.text=="NO":
       bot.send_message(message.chat.id,"<b>Su cita para envio de mercancia a sido cancelado</b>",parse_mode="HTML") 
usuarios=pd.DataFrame(usuarios)   
@bot.message_handler(content_types=["text"])
def bot_mensaje_texto(message):
  if message.text.startswith("/"):
    bot.send_message(message.chat.id, "Comando no disponible")
  else:
    bot.send_message(message.chat.id, "Introduce un comando ")




In [258]:
def cot_menu(message):
    if message.text == "1":
        bot.send_message(message.chat.id,  "El costo por envío es de $ "+costo_estandar_por_envio)
    elif message.text == "2":
        bot.send_message(message.chat.id, "El costo por dimensiones de paquete de envío es de $ ")
    elif message.text == "3":
        bot.send_message(message.chat.id, "Los métodos de pago disponibles son tarjeta de crédito, transferencia bancaria y PayPal.")
    elif message.text == "4":
        bot.send_message(message.chat.id, "Volviendo")
        start(message)
    else:
        bot.reply_to(message, "Opción no válida. Vuelva a intentarlo.")


In [259]:
def envios_submenu(message):
    bot.send_message(message.chat.id, "Saliendo al menu principal")
    if message.text == "1":
        bot.send_message(message.chat.id, "Paquete en buen estado.")
    elif message.text == "2":
        bot.send_message(message.chat.id, "Embarque")
    elif message.text == "3":
        bot.send_message(message.chat.id, "Jorde Mendoza.")
    else:
        bot.send_message(message.chat.id, "Opción no válida, vuelve a intentarlo")

In [ ]:
@bot.message_handler(func=lambda message: message.text == 'Contactar Agente')
def contact_agent(message):
    bot.send_message(message.chat.id, "Envia tu nombre, email y  consulta.")


In [ ]:
    bot.register_next_step_handler(message, handle_contact_agent_info)


In [ ]:
def handle_contact_agent_info(message):
    name, email, inquiry = message.text.split(',')
    # Enviar información al agente 
    bot.send_message(message.chat.id, f"Gracia {name}. Nuetro agente te contactará pronto.")


In [ ]:
bot.infinity_polling()